In [1]:
def corr_kg_fob_via_preço(comex_df,z=3):
    
    col_log=comex_df[['log_kg','log_fob']].columns
    
    def z_score(comex_df,l):
        média=comex_df[l].mean()
        k=len(comex_df.columns)
        comex_df.insert(k,'mean_'+l,média)
        comex_df.insert(k+1,'qd_dif_'+l,(comex_df[l]-comex_df['mean_'+l])**2)
        comex_df.insert(k+2,'std_'+l, (comex_df['qd_dif_'+l].sum()/len(comex_df))**(1/2))
        comex_df.insert(k+3,'z_score_'+l, (comex_df[l]-comex_df['mean_'+l])/comex_df['std_'+l])
        
        return comex_df
    
    for l in col_log:
        comex_df=z_score(comex_df,l)

    k=len(comex_df.columns)

    comex_df.insert(k,
                    'dif_z_score',
                    comex_df['z_score_'+col_log[0]] - comex_df['z_score_'+col_log[1]]
                   )
    
    comex_df=z_score(comex_df,'dif_z_score')
    
    comex_df_corr=comex_df[np.abs(comex_df.z_score_dif_z_score)<=z]
    comex_df_err=comex_df[np.abs(comex_df.z_score_dif_z_score)>z]
    
    preço=comex_df_corr['US$_fob'].sum()/comex_df_corr.kg.sum()
    
    comex_df_err=comex_df_err.assign(preço=preço)
    
    comex_df_err=comex_df_err.assign(kg_err=1)
    
    comex_df_err.loc[np.abs(comex_df_err['z_score_log_fob'])>=np.\
                     abs(comex_df_err.z_score_log_kg),'kg_err']=0
    
    comex_df_err_kg=comex_df_err[comex_df_err.kg_err==1]
    
    comex_df_err_fob=comex_df_err[comex_df_err.kg_err==0]
    
    comex_df_err_kg.kg=comex_df_err_kg['US$_fob']/preço

    len_err_kg=len(comex_df_err_kg)
    
    comex_df_err_fob['US$_fob']=comex_df_err_fob.kg*preço
    
    len_err_fob=len(comex_df_err_fob)
    
    comex_df=pd.concat([comex_df_corr,comex_df_err_kg,comex_df_err_fob])
    
    return comex_df, len_err_kg, len_err_fob

print('** Função corr_kg_fob_via_preço(comex_df,z=3) disponível **')

** Função corr_kg_fob_via_preço(imp_fert,z=3) disponível **
